In [15]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from imblearn.over_sampling import SMOTE, SMOTENC

In [2]:
from sklearn.model_selection import RandomizedSearchCV

In [21]:
X_test = pd.read_csv('X_test_categories.csv')
X_test.head()

,race,gender,age,admission_type_id,discharge_disposition_id,time_in_hospital,payer_code,num_medications,diag_1,diag_2,...,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,total_treatments,total_visits,total_procedures
0,Caucasian,Female,5,Emergency,Home,2,Unknown_Other,4,diabetes,Other,...,0,0,0,0,0,No,Yes,1,0,33
1,Caucasian,Male,75,Other,Home,2,Medicare,16,Other,Other,...,0,0,0,0,0,Yes,Yes,2,1,33
2,Caucasian,Female,65,Other,Other,5,Unknown_Other,24,diabetes,circulatory,...,0,0,0,0,0,No,Yes,1,0,50
3,Caucasian,Female,65,Other,Home,5,Unknown_Other,16,circulatory,circulatory,...,0,0,0,0,0,No,Yes,1,1,33
4,Caucasian,Male,35,Other,Home,3,Commericial/Other,4,injury,Other,...,0,0,0,0,0,No,No,0,0,33


In [22]:
X_train = pd.read_csv('X_train_categories.csv')
X_train.head()

,race,gender,age,admission_type_id,discharge_disposition_id,time_in_hospital,payer_code,num_medications,diag_1,diag_2,...,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,total_treatments,total_visits,total_procedures
0,Caucasian,Female,65,Emergency,HomeHealth,14,Unknown_Other,59,circulatory,circulatory,...,0,0,0,0,0,Yes,Yes,2,0,71
1,Caucasian,Male,25,Emergency,Home,6,Unknown_Other,21,Other,Other,...,0,0,0,0,0,Yes,Yes,1,0,73
2,AfricanAmerican,Female,65,Other,Home,10,Unknown_Other,12,Other,circulatory,...,0,0,0,0,0,No,Yes,1,1,40
3,Caucasian,Male,45,Emergency,Home,1,Commericial/Other,16,diabetes,digestive,...,0,0,0,0,0,Yes,Yes,2,4,38
4,Caucasian,Male,65,Emergency,Home,2,Medicare,14,respiratory,respiratory,...,0,0,0,0,0,No,Yes,1,1,51


In [23]:
y_test = pd.read_csv('multi_y_test.csv')
y_test.head()

,readmitted_multi
0,0
1,0
2,2
3,1
4,0


In [24]:
y_train = pd.read_csv('multi_y_train.csv')
y_train.head()

,readmitted_multi
0,0
1,0
2,2
3,1
4,0


In [25]:
X_train.head()

,race,gender,age,admission_type_id,discharge_disposition_id,time_in_hospital,payer_code,num_medications,diag_1,diag_2,...,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,total_treatments,total_visits,total_procedures
0,Caucasian,Female,65,Emergency,HomeHealth,14,Unknown_Other,59,circulatory,circulatory,...,0,0,0,0,0,Yes,Yes,2,0,71
1,Caucasian,Male,25,Emergency,Home,6,Unknown_Other,21,Other,Other,...,0,0,0,0,0,Yes,Yes,1,0,73
2,AfricanAmerican,Female,65,Other,Home,10,Unknown_Other,12,Other,circulatory,...,0,0,0,0,0,No,Yes,1,1,40
3,Caucasian,Male,45,Emergency,Home,1,Commericial/Other,16,diabetes,digestive,...,0,0,0,0,0,Yes,Yes,2,4,38
4,Caucasian,Male,65,Emergency,Home,2,Medicare,14,respiratory,respiratory,...,0,0,0,0,0,No,Yes,1,1,51


In [10]:
# for idx, col in enumerate(X_train.columns):
#     print(idx, col)

In [26]:
cats_idx = [0, 1, 3, 4, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20,
           21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]

In [27]:
smnc = SMOTENC(
    sampling_strategy='auto', # samples only the minority class
    random_state=0,  
    k_neighbors=5,
    n_jobs=4,
    categorical_features=cats_idx
)  

X_res, y_res = smnc.fit_resample(X_train, y_train)

In [36]:
X_res, y_res = X_train, y_train

In [28]:
print(f'After OverSampling, the shape of train_X: {X_res.shape}') 
print(f'After OverSampling, the shape of train_y: {y_res.shape} \n') 
 
print(f'After OverSampling, counts of label 2 : {len(y_res == 2)}')     
print(f'After OverSampling, counts of label 1 : {len(y_res == 1)}') 
print(f'After OverSampling, counts of label 0 : {len(y_res == 0)}') 

After OverSampling, the shape of train_X: (127617, 40)
After OverSampling, the shape of train_y: (127617, 1) 

After OverSampling, counts of label 2 : 127617
After OverSampling, counts of label 1 : 127617
After OverSampling, counts of label 0 : 127617


In [30]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import recall_score, f1_score, accuracy_score
from sklearn.metrics import precision_score, make_scorer
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [45]:
def model_test(model, X_train, y_train, X_test, y_test):
   # fit model on training data
    model.fit(X_train, y_train)
    
    # get model predictions and score on test set
    predictions = model.predict(X_test)
    score = model.score(X_test, y_test)
    return predictions, score


In [44]:
def get_model_performance(predictions, color, X_test, y_test):
    cfm = confusion_matrix(y_test, predictions)
    clr = classification_report(y_test, predictions, 
                                target_names = ['No Readmission', 'Readmission'])
                                           
    true_negative = cfm[0][0]
    false_positive = cfm[0][1]
    false_negative = cfm[1][0]
    true_positive = cfm[1][1]

    precision = precision_score(y_test, predictions, average='macro')
    recall = recall_score(y_test, predictions, average='macro')
    f1 = f1_score(y_test, predictions, average='macro')
    accuracy = accuracy_score(y_test, predictions)
    
    print(f'Confusion Matrix: \n', cfm, '\n')
    print(f'Classification Report: \n', clr, '\n')
    
    print(f'True Negative: {true_negative}')
    print(f'False Positive: {false_positive}')
    print(f'False Negative: {false_negative}')
    print(f'True Positive: {true_positive}', '\n')
    
    print (f'Precision score: {precision}')
    print (f'Recall score: {recall}')
    print (f'F1 score : {f1}')
    print (f'Accuracy score: {accuracy}')
    
    plt.figure(figsize=(13,8))
    sns.heatmap(confusion_matrix(y_test, predictions), 
            annot=True, fmt=".3f", linewidths=.5, square = True, 
                cmap = color);
    plt.ylabel('Actual label');
    plt.xlabel('Predicted label');
    all_sample_title = (f'Recall Score: {recall}')
    plt.title(all_sample_title, size = 15);

In [43]:
def get_important_feats(model, X_train, X_test, y_test):
    feature_importance = model.feature_importances_
    feat_importances = pd.Series(model.feature_importances_, 
                                 index=X_train.columns)
    feat_importances = feat_importances.nlargest(19)
    feat_importances.plot(kind='barh' , figsize=(10,10)) 
    plt.show()

In [41]:
xgb = XGBClassifier(objective='multi:softmax', num_class=3,
                           eval_metric = ["logloss", "auc"])
decision = DecisionTreeClassifier()
gbc = GradientBoostingClassifier()
knn = KNeighborsClassifier()
forest = RandomForestClassifier()

In [46]:
predictions, score = model_test(xgb, X_train, y_train, X_test, y_test)
get_model_performance(predictions, 'Blues')

get_important_feats(xgb, X_train)

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields race, gender, admission_type_id, discharge_disposition_id, payer_code, diag_1, diag_2, diag_3, max_glu_serum, A1Cresult, insulin, change, diabetesMed